# Preprocessing CyGNSS data

Data is downloaded from NASA EarthCloud as described in the `APIs` notebook. For the expected format for CyGNSSnet, additional preprocessing steps are necessary.

In [1]:
import os
import sys
sys.path.append('/work/ka1176/caroline/gitlab/2020-03-gfz-remote-sensing/')

In [2]:
from gfz_202003.preprocessing import preprocess as prep

In [3]:
import numpy as np
import h5py
from matplotlib import pyplot as plt
import seaborn as sns

import datetime
import xarray as xr

import argparse

In [4]:
import cdsapi

## Download raw CyGNSS data

The CyGNSSnet preprocessing routine expects the raw data files ordered as 

> `$raw_data_dir/<year>/<day-of-year>/cyg*.nc`

Data is always downloaded for one full day for all spacecraft, generating 8 `netcdf` files per day of observations. Below is a routine to specify a date range, followed by downloading the corresponding data and storing it in the appropriate subfolders.

In [5]:
raw_data_root = '/work/ka1176/shared_data/2022-cygnss-deployment/raw_data/'
dev_data_root = '/work/ka1176/shared_data/2022-cygnss-deployment/dev_data/'

Select a test day and prepared the input parameters for the provided download script

In [6]:
year  = 2021
month = 3
day   = 17

Downloaded target directory in the expected format `year/day-of-year`

In [7]:
raw_data_sub = datetime.datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d").strftime("%Y/%j")

raw_data_dir = os.path.join(raw_data_root, raw_data_sub)

print(raw_data_dir)

/work/ka1176/shared_data/2022-cygnss-deployment/raw_data/2021/076


Start and end date of download range in the required format. The end date is midnight the next day, this way only the requested day's data is downloaded.

In [8]:
start_date = datetime.datetime(year, month, day).strftime("%Y-%m-%dT%H:%M:%SZ")
end_date   = datetime.datetime(year, month, day + 1).strftime("%Y-%m-%dT%H:%M:%SZ")

print(f'--start-date {start_date}')
print(f'--end-date   {end_date}')

--start-date 2021-03-17T00:00:00Z
--end-date   2021-03-18T00:00:00Z


In [9]:
dday = datetime.datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d").strftime("%j") # need that later
dday

'076'

In [10]:
%env PYTHONPATH=/work/ka1176/caroline/gitlab/data-subscriber
!python /work/ka1176/caroline/gitlab/data-subscriber/subscriber/podaac_data_downloader.py  -c CYGNSS_L1_V3.1 -d $raw_data_dir --start-date $start_date --end-date $end_date

env: PYTHONPATH=/work/ka1176/caroline/gitlab/data-subscriber
[2022-09-02 08:42:26,045] {podaac_data_downloader.py:155} INFO - NOTE: Making new data directory at /work/ka1176/shared_data/2022-cygnss-deployment/raw_data/2021/076(This is the first run.)
[2022-09-02 08:42:26,664] {podaac_data_downloader.py:242} INFO - Found 8 total files to download
[2022-09-02 08:43:09,912] {podaac_data_downloader.py:273} INFO - 2022-09-02 08:43:09.912371 SUCCESS: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/CYGNSS_L1_V3.1/cyg07.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc
[2022-09-02 08:43:50,828] {podaac_data_downloader.py:273} INFO - 2022-09-02 08:43:50.828714 SUCCESS: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/CYGNSS_L1_V3.1/cyg03.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc
[2022-09-02 08:44:48,103] {podaac_data_downloader.py:273} INFO - 2022-09-02 08:44:48.102970 SUCCESS: https://archive.podaac.earthdata.nasa

## Download raw ERA5 data

The preprocessing pipeline requires the ERA5 windspeed labels. Download the raw ERA5 data for the same timespan.

In [11]:
era5_data = os.path.join(raw_data_dir, 'ERA5_windspeed.nc')

In [12]:
cds = cdsapi.Client()

In [13]:
cds.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind',
        ],
        'year': year,
        'month': month,
        'day': day,
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00'
        ],
        'area': [
            40, -180, -40, 180,
        ],
    },
    era5_data)

/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-09-02 08:48:40,317 INFO Welcome to the CDS
2022-09-02 08:48:40,317 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-09-02 08:48:40,463 INFO Request is queued
/sw/spack-levante/mambaforge-4.11.0-0-Linu

Result(content_length=44383516,content_type=application/x-netcdf,location=https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data7/adaptor.mars.internal-1662101323.2529619-2948-3-0f4e6034-427d-4e4a-89f3-cf4143873fa3.nc)

In [14]:
era5_ds = xr.open_dataset(era5_data)
era5_ds

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 321, time: 24)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 40.0 39.75 39.5 39.25 ... -39.5 -39.75 -40.0
  * time       (time) datetime64[ns] 2021-03-17 ... 2021-03-17T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-02 06:48:43 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

## Annotate raw CyGNSS data with windspeed labels

We need to create the data variables `ERA5_u10` and `ERA5_v10` and attach them to the CyGNSS raw data.

In [15]:
os.listdir(raw_data_dir)

['cyg01.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg08.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg02.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'ERA5_windspeed.nc',
 'cyg03.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg05.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg06.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg07.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg04.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc']

Check units for spacetime coordinates
* Longitude
  * ERA5:   -180 ... 0 ... +180
  * CyGNSS: 0 ... 180 ... 360
* Latitude
  * ERA5 & CyGNSS: -40 ... 0 ... +40
* Timestamp


--> Need to shift the ERA5 longitude coordinate by 180

In [17]:
def annotate_dataset(cygnss_file, era5_file, save_dataset=False):
    '''
    Annotate a given CyGNSS dataset with ERA5 windspeed labels and save to disk
    
    Parameters:
    cygnss_file : path to CyGNSS dataset
    era5_file   : path to orresponding ERA5 dataset
    save_dataset : if True, save dataset to disk overwriting cygnss_file (default: False)
    
    Returns:
    Annotated CyGNSS dataset
    '''
    
    # necessary because lazy loading prohibits overwriting the netcdf files at the end of this section
    with xr.open_dataset(cygnss_file) as data:
        cygnss_ds = data.load()
        
    with xr.open_dataset(era5_file) as data:
        era5_ds = data.load()
        
    # needs to be shifted by 180 for compatibility with CyGNSS
    era5_ds = era5_ds.assign_coords(longitude=era5_ds.coords['longitude'] + 180)
    
    interp_ds = era5_ds.interp(longitude=cygnss_ds.sp_lon, latitude=cygnss_ds.sp_lat, time=cygnss_ds.ddm_timestamp_utc)
    
    cygnss_ds['ERA5_u10'] = interp_ds['u10']
    cygnss_ds['ERA5_v10'] = interp_ds['v10']

    tmp_attrs = cygnss_ds['ERA5_u10'].attrs
    tmp_attrs['long_name'] = cygnss_ds['ERA5_u10'].long_name + ' (interpolated)'
    cygnss_ds['ERA5_u10'].attrs = tmp_attrs

    tmp_attrs = cygnss_ds['ERA5_v10'].attrs
    tmp_attrs['long_name'] = cygnss_ds['ERA5_v10'].long_name + ' (interpolated)'
    cygnss_ds['ERA5_v10'].attrs = tmp_attrs
    
    cygnss_ds = cygnss_ds.drop_vars(['longitude', 'latitude', 'time'])
    
    # dummy values only for preprocessing routine
    cygnss_ds['GPM_precipitation'] = -9999
    cygnss_ds['ERA5_mdts'] = -9999
    cygnss_ds['ERA5_mdww'] = -9999
    cygnss_ds['ERA5_swh'] = -9999
    cygnss_ds['ERA5_shts'] = -9999
    cygnss_ds['ERA5_shww'] = -9999
    cygnss_ds['ERA5_p140121'] = -9999
    cygnss_ds['ERA5_p140124'] = -9999
    cygnss_ds['ERA5_p140127'] = -9999
    
    if save_dataset:
        cygnss_ds.to_netcdf(cygnss_file)
        
    return cygnss_ds

In [18]:
for cygnss_file in os.listdir(raw_data_dir):
    if cygnss_file.startswith('cyg') and cygnss_file.endswith('.nc'):
        print(cygnss_file)
        annotate_dataset(os.path.join(raw_data_dir, cygnss_file), era5_data, save_dataset=True)

cyg01.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc
cyg08.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc
cyg02.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc
cyg03.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc
cyg05.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc
cyg06.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc
cyg07.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc
cyg04.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc


## Check raw data

In [19]:
from importlib import reload
reload(prep)
raw_ds = prep.open_mfdataset(os.path.join(raw_data_dir, cygnss_file))

raw_ds

<xarray.Dataset>
Dimensions:                          (sd: 691096, delay: 17, doppler: 11)
Coordinates:
  * sd                               (sd) MultiIndex
  - sample                           (sd) int64 0 0 0 0 ... 172773 172773 172773
  - ddm                              (sd) int64 0 1 2 3 0 1 2 ... 1 2 3 0 1 2 3
Dimensions without coordinates: delay, doppler
Data variables: (12/152)
    spacecraft_id                    (sd) int16 44 44 44 44 44 ... 44 44 44 44
    spacecraft_num                   (sd) int8 4 4 4 4 4 4 4 4 ... 4 4 4 4 4 4 4
    ddm_source                       (sd) int8 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2
    ddm_time_type_selector           (sd) int8 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    delay_resolution                 (sd) float32 0.2552 0.2552 ... 0.2552
    dopp_resolution                  (sd) float32 500.0 500.0 ... 500.0 500.0
    ...                               ...
    les_scatter_area_log10           (sd) float32 dask.array<chunksize=(691096,), meta=np.ndarray>
    ddm_timestamp_unix               (sd) float64 dask.array<chunksize=(691096,), meta=np.ndarray>
    ddm_nbrcs_exceeds_alpha_0.95     (sd) bool dask.array<chunksize=(691096,), meta=np.ndarray>
    ddm_nbrcs_exceeds_alpha_0.05     (sd) bool dask.array<chunksize=(691096,), meta=np.ndarray>
    ddm_nbrcs_exceeds_alpha_0.975    (sd) bool dask.array<chunksize=(691096,), meta=np.ndarray>
    ddm_nbrcs_exceeds_alpha_0.025    (sd) bool dask.array<chunksize=(691096,), meta=np.ndarray>
Attributes: (12/66)
    Conventions:                          CF-1.6, ACDD-1.3, ISO-8601
    standard_name_vocabulary:             CF Standard Name Table v30
    project:                              CYGNSS
    featureType:                          trajectory
    summary:                              CYGNSS is a NASA Earth Venture miss...
    program:                              CYGNSS
    ...                                   ...
    geospatial_lon_max:                   359.999E
    time_coverage_resolution:             P0DT0H0M1S
    time_coverage_start:                  2021-03-17T00:00:00.499261784Z
    time_coverage_end:                    2021-03-17T23:59:59.999261412Z
    time_coverage_duration:               P1DT0H0M0S
    NCO:                                  4.4.4

In [20]:
filtered_ds = prep.apply_quality_filter(raw_ds, is_ml_ops=True)
filtered_ds

<xarray.Dataset>
Dimensions:                          (sd: 2747, delay: 17, doppler: 11)
Coordinates:
  * sd                               (sd) MultiIndex
  - sample                           (sd) int64 4426 4427 4428 ... 151629 163455
  - ddm                              (sd) int64 0 0 0 0 0 0 0 ... 0 2 2 3 2 3 3
Dimensions without coordinates: delay, doppler
Data variables: (12/152)
    spacecraft_id                    (sd) int16 44 44 44 44 44 ... 44 44 44 44
    spacecraft_num                   (sd) int8 4 4 4 4 4 4 4 4 ... 4 4 4 4 4 4 4
    ddm_source                       (sd) int8 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2
    ddm_time_type_selector           (sd) int8 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    delay_resolution                 (sd) float32 0.2552 0.2552 ... 0.2552
    dopp_resolution                  (sd) float32 500.0 500.0 ... 500.0 500.0
    ...                               ...
    les_scatter_area_log10           (sd) float32 dask.array<chunksize=(2747,), meta=np.ndarray>
    ddm_timestamp_unix               (sd) float64 dask.array<chunksize=(2747,), meta=np.ndarray>
    ddm_nbrcs_exceeds_alpha_0.95     (sd) bool dask.array<chunksize=(2747,), meta=np.ndarray>
    ddm_nbrcs_exceeds_alpha_0.05     (sd) bool dask.array<chunksize=(2747,), meta=np.ndarray>
    ddm_nbrcs_exceeds_alpha_0.975    (sd) bool dask.array<chunksize=(2747,), meta=np.ndarray>
    ddm_nbrcs_exceeds_alpha_0.025    (sd) bool dask.array<chunksize=(2747,), meta=np.ndarray>
Attributes: (12/66)
    Conventions:                          CF-1.6, ACDD-1.3, ISO-8601
    standard_name_vocabulary:             CF Standard Name Table v30
    project:                              CYGNSS
    featureType:                          trajectory
    summary:                              CYGNSS is a NASA Earth Venture miss...
    program:                              CYGNSS
    ...                                   ...
    geospatial_lon_max:                   359.999E
    time_coverage_resolution:             P0DT0H0M1S
    time_coverage_start:                  2021-03-17T00:00:00.499261784Z
    time_coverage_end:                    2021-03-17T23:59:59.999261412Z
    time_coverage_duration:               P1DT0H0M0S
    NCO:                                  4.4.4

In [21]:
os.listdir('/work/ka1176/shared_data/2020-03/raw_data/2021/014/')

['cyg05.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc.md5',
 'cyg01.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc.md5',
 'cyg06.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc',
 'cyg02.ddmi.s20210114-011822-e20210114-235959.l1.power-brcs.a30.d31.nc.md5',
 'cyg05.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc',
 'cyg03.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc.md5',
 'cyg04.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc.md5',
 'cyg02.ddmi.s20210114-011822-e20210114-235959.l1.power-brcs.a30.d31.nc',
 'cyg07.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc',
 'cyg06.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc.md5',
 'cyg01.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc',
 'cyg08.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc',
 'cyg04.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc',
 'cyg08.ddmi.s

In [22]:
bu = raw_ds['ddm_brcs_uncert']
qf = raw_ds['quality_flags']
st = raw_ds['nst_att_status']
fom = raw_ds['prn_fig_of_merit']
les = raw_ds['ddm_les']
rxg = raw_ds['sp_rx_gain']
nsca = raw_ds['nbrcs_scatter_area']
lsca = raw_ds['les_scatter_area']
lat = raw_ds['sp_lat']
lon = raw_ds['sp_lon']
ws = raw_ds['windspeed']

For now, use only the quality flag == 4

In [23]:
quality = (bu<1) & (qf == 4) & (st == 0) & (fom > 3) & (rxg > 0) & (les >= 0)

In [24]:
np.sum((bu<1) & (st==0)).compute()

<xarray.DataArray ()>
array(184613)

## Created processed data

In [25]:
raw_ds = prep.open_mfdataset(os.path.join(raw_data_dir, 'cyg06*.nc'), channels=[0,1,2,3])

In [26]:
dev_data_dir = '/work/ka1176/shared_data/2022-cygnss-deployment/dev_data/'

In [27]:
for ff in os.listdir('/work/ka1176/shared_data/2022-cygnss-deployment/raw_data/2021/168/'):
    tmp = xr.open_dataset(os.path.join('/work/ka1176/shared_data/2022-cygnss-deployment/raw_data/2021/168/', ff))
    if not 'ERA5_u10' in tmp.keys():
        print(ff)

ERA5_windspeed.nc


In [28]:
tmp

<xarray.Dataset>
Dimensions:                          (sample: 172776, ddm: 4, delay: 17,
                                      doppler: 11)
Coordinates:
  * sample                           (sample) int32 0 1 2 ... 172774 172775
  * ddm                              (ddm) int8 0 1 2 3
    ddm_timestamp_utc                (sample) datetime64[ns] ...
    sp_lat                           (sample, ddm) float32 ...
    sp_lon                           (sample, ddm) float32 ...
Dimensions without coordinates: delay, doppler
Data variables: (12/137)
    spacecraft_id                    int16 ...
    spacecraft_num                   int8 ...
    ddm_source                       int8 ...
    ddm_time_type_selector           int8 ...
    delay_resolution                 float32 ...
    dopp_resolution                  float32 ...
    ...                               ...
    ERA5_swh                         int64 ...
    ERA5_shts                        int64 ...
    ERA5_shww                        int64 ...
    ERA5_p140121                     int64 ...
    ERA5_p140124                     int64 ...
    ERA5_p140127                     int64 ...
Attributes: (12/66)
    Conventions:                          CF-1.6, ACDD-1.3, ISO-8601
    standard_name_vocabulary:             CF Standard Name Table v30
    project:                              CYGNSS
    featureType:                          trajectory
    summary:                              CYGNSS is a NASA Earth Venture miss...
    program:                              CYGNSS
    ...                                   ...
    geospatial_lon_max:                   360.000E
    time_coverage_resolution:             P0DT0H0M1S
    time_coverage_start:                  2021-06-17T00:00:00.499261472Z
    time_coverage_end:                    2021-06-17T23:59:59.999261780Z
    time_coverage_duration:               P1DT0H0M0S
    NCO:                                  4.4.4

In [29]:
reload(prep)
args = argparse.Namespace(raw_data_dir='/work/ka1176/shared_data/2022-cygnss-deployment/raw_data/',
                          output_dir=dev_data_dir,
                          v_map=['brcs'],
                          n_valid_days=0,
                          n_test_days=1,
                          n_processes=1,
                          only_merge=False,
                          use_land_data=False,
                          is_ml_ops=True,
                          version='v3.1',
                          day=dday,
                          year=year,
                          reduce_mode='')

prep.generate_input_data(args)

starting with day 076 of split test
processing year 2021, day 076, day counter 807, at /work/ka1176/shared_data/2022-cygnss-deployment/raw_data/2021/076/cyg*.nc


/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/core/dataset.py:3929: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._stack_once(dims, new_dim)
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/core/dataset.py:3929: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ... 

15209 samples of day 076 appended to file /work/ka1176/shared_data/2022-cygnss-deployment/dev_data/807.h5 in 0.54 minutes
samples of /work/ka1176/shared_data/2022-cygnss-deployment/dev_data/807.h5 appended to /work/ka1176/shared_data/2022-cygnss-deployment/dev_data/test_data.h5 in 0.17 seconds


## Check the new CyGNSS data v3.1

In [30]:
TODO annotate the samples with date (year month day etc)

SyntaxError: invalid syntax (3716742565.py, line 1)

In [ ]:
!conda list env